In [ ]:
"""
Tumor Classsification using VGG-19
Author: Amruth Karun M V
Date: 07-Nov-2021
"""

import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Dense, Input, Dropout, Flatten, 
    Conv2D, MaxPooling2D, BatchNormalization
)

from sklearn import metrics
import matplotlib.pyplot as plt
%matplotlib inline

TRAIN_PATH = "../input/brain-tumor-mri-dataset/Training"
TEST_PATH = "../input/brain-tumor-mri-dataset/Testing"
CLASS_NAMES = ['Glioma', 'Meningioma', 'No-tumor', 'Pituitary']
EPOCHS = 100
BATCH_SIZE = 128
LEARNING_RATE = 0.001


def plot_sample_images():
    """
    Plots sample images for each class
    Arguments: None
    Returns: Plots sample data
    """
    
    plt.figure(figsize=(10, 10))
    sample_image_path = ['/glioma/Tr-gl_0010.jpg', '/meningioma/Tr-me_0010.jpg', 
                         '/notumor/Tr-no_0010.jpg', '/pituitary/Tr-pi_0010.jpg']
    for i in range(len(CLASS_NAMES)):
        ax = plt.subplot(2, 2, i + 1)
        img = cv2.imread(TRAIN_PATH + sample_image_path[i])
        img = cv2.resize(img, (128, 128))
        plt.imshow(img)
        plt.title(CLASS_NAMES[i])
    
    
def load_data(input_path, shuffle=False):
    """
    Loads input data fro directory
    Arguments:
        input_path -- input data path
        shuffle    -- whether data needs to be shuffled or not
    Returns: Data generator
    """
    
    data_generator = keras.preprocessing.image.ImageDataGenerator()
    data_generator = data_generator.flow_from_directory(directory=input_path, target_size=(224,224), 
                                                        shuffle=shuffle, class_mode= "categorical")
    
    return data_generator


def load_model():
    """
    Creates a keras VGG-19 model
    Arguments: None
    Returns: VGG-19 Model
    """
    
    img_input = Input(shape=(224, 224, 3))
    
    # Block 1
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv4')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv4')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    # Block 5
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv4')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)
    
    # Classification block
    x = Flatten(name='flatten')(x)
    x = Dense(4096, activation='relu', name='fc1')(x)
    x = Dropout(0.4)(x)
    x = Dense(4096, activation='relu', name='fc2')(x)
    x = Dropout(0.4)(x)
    x = Dense(4, activation='softmax', name='predictions')(x)
    
    model = Model(img_input, x, name='vgg-19')
    
    model.summary()
   
    opt = Adam(learning_rate=LEARNING_RATE)
    model.compile(loss = keras.losses.categorical_crossentropy, optimizer=opt, metrics=['accuracy'])
    
    return model


def plot_curves(history):
    """
    Plots loss and accuracy and loss plots for
    training and validation datasets
    Arguments: 
        history -- training history
    Returns: None
    """
   
    plt.plot(history.history['loss'], label="Training loss")
    plt.plot(history.history['val_loss'], label="Validation loss")
    plt.legend()
    plt.title('Training Loss VS Validation Loss')
    plt.show()
    
    plt.plot(history.history['accuracy'], label="Training accuracy")
    plt.plot(history.history['val_accuracy'], label="Validation accuracy")
    plt.title('Training Accuracy VS Validation Accuracy')
    plt.legend()
    plt.show()
    

def evaluate_model(model, input_path):
    """
    Evaluates the model and displays
    the confusion matrix
    Arguments:
        model       -- trained model
        input_path  -- input data path
    Returns: Model score and confusion matrix
    """

    data_generator = load_data(input_path)
    predictions = model.predict(data_generator, BATCH_SIZE)
    y_pred = np.argmax(predictions, axis=1)
    y_true = data_generator.classes
    
    print("Score = ", model.evaluate(data_generator))
    print("Accuracy = ", metrics.accuracy_score(y_true, y_pred))
    cm = metrics.confusion_matrix(y_true, y_pred)
    metrics.ConfusionMatrixDisplay(cm, display_labels=CLASS_NAMES).plot(cmap=plt.cm.Blues,
                                                                       xticks_rotation='vertical')
    plt.show()

    
def train_model():
    """
    Trains VGG-19 model and saves the 
    trained weights to an H5 file.
    Arguments: None
    Returns: None
    """
    
    train_generator = load_data(TRAIN_PATH, True)
    val_generator = load_data(TEST_PATH, True)
    
    # Loads VGG-19 model
    model = load_model()
    earlystop = keras.callbacks.EarlyStopping(monitor='loss', min_delta=1e-11, patience=10)
    reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, 
                                                   patience=6, verbose=1)
    model_callbacks = [earlystop, reduce_lr]
    
    history = model.fit(
        train_generator, 
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        validation_data=val_generator,
        validation_steps=val_generator.samples//BATCH_SIZE,
        steps_per_epoch=train_generator.samples//BATCH_SIZE,
        callbacks=model_callbacks)
    
    plot_curves(history)
    model.save_weights("model_vgg19.h5")
    print("Model saved successfully!")
    
    return model


In [ ]:
plot_sample_images()

In [ ]:
# Train the model
model = train_model()
print("Confusion matrix for train data: ")
evaluate_model(model, TRAIN_PATH)
print("Confusion matrix for val data: ")
evaluate_model(model, TEST_PATH)